# An fMRI study of composition in noun and verb phrases
### Bonnasse-Gahot, Bemis, Perez-Guevara, Dehaene, Pallier
Generate the events for the anova model

In [1]:
import numpy as np
import pandas as pd
import os.path as op
import simpcomp as sc

In [2]:
# regressors are:
# pn1, pn2, pn3, pn4 : noun phrases of length 1, 2, 3 and 4 respectively
# pv1, pv2, pv3, pv4 : verb phrases of length 1, 2, 3 and 4 respectively
# ln1, ln2, ln3, ln4 : noun lists of length 1, 2, 3 and 4 respectively
# lv1, lv2, lv3, lv4 : verb lists of length 1, 2, 3 and 4 respectively
# probe
# resp_r : response key right
# resp_l : response key left 

def create_model_events(full_events):
    model_events = pd.DataFrame(columns=['onset', 'duration', 'trial_type'])   

    for trial_number in range(1,81):
        onset = full_events[full_events.trial_number==trial_number].onset.iloc[1]
        duration = (full_events[full_events.trial_number==trial_number].onset.iloc[10]
                    - full_events[full_events.trial_number==trial_number].onset.iloc[1])
        trial_type = '{}{}{}'.format(full_events[full_events.trial_number==trial_number].trial_structure.iloc[0][0],
                                     full_events[full_events.trial_number==trial_number].trial_type.iloc[0][0],
                                     full_events[full_events.trial_number==trial_number].trial_length.iloc[0])
        model_events = model_events.append(pd.Series({'onset':onset,
                                                      'duration':duration,
                                                      'trial_type':trial_type}),
                                           ignore_index=True)

        onset = full_events[full_events.trial_number==trial_number].iloc[12].onset
        duration = 0
        trial_type = 'probe'
        model_events = model_events.append(pd.Series({'onset':onset,
                                                      'duration':duration,
                                                      'trial_type':trial_type}),
                                           ignore_index=True)  

        if full_events[full_events.trial_number==trial_number].response_key.iloc[0] != 'TIMEOUT':
            onset = full_events[(full_events.trial_number==trial_number)&(full_events.stim=='[response]')].onset.iloc[0]
            duration = 0
            if full_events[full_events.trial_number==trial_number].response_key.iloc[0] == 'y':
                trial_type = 'resp_r'
            else:
                trial_type = 'resp_l'
            model_events = model_events.append(pd.Series({'onset':onset,
                                                          'duration':duration,
                                                          'trial_type':trial_type}),
                                               ignore_index=True)  
    return model_events

In [3]:
for sub_id in sc.subject_list:
    dir_tmp = op.join(sc.model_anova_folder, 'events', sub_id)
    sc.make_dir(dir_tmp)
    for indrun in range(1,5):
        filename_in = op.join(sc.events_folder, sub_id, 'func',
                   '{}_task-maindesign_run-{:02d}_events.tsv'.format(sub_id, indrun))
        full_events = pd.read_csv(filename_in, sep='\t')
        model_events = create_model_events(full_events)
        
        filename_out = op.join(dir_tmp, 
                      '{}_model-anova_run-{:02d}_events.tsv'.format(sub_id, indrun))
        model_events.to_csv(filename_out, index=False, sep='\t')